In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import pysam
import numpy as np
import cPickle as pickle
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from collections import Counter
from random import randint

In [4]:
import logging
logging.basicConfig(level=logging.INFO)

In [5]:
def assign_color():
    
    return (str(randint(0, 255)) + ',' + str(randint(0, 255)) + ',' + str(randint(0, 255)))

In [6]:
def color_reads(in_file):
    
    st = pysam.AlignmentFile(in_file,"rb")
    st.reset()
    reads = st.fetch(until_eof=True)
    
    color_map = {}
    for r in reads:
        if not r.query_name in color_map:
            color_map.update({r.query_name:assign_color()})
    
    out_file = "%s.colored.bam" % os.path.splitext(in_file)[0]
    colored_reads = pysam.AlignmentFile('tmp.bam', 'wb', template=st)
    
    st.reset()
    reads = st.fetch(until_eof=True)
    for r in reads:
        if r.has_tag('RG'):
            r.set_tag('RG', None)
        color = color_map[r.query_name]
        r.tags += [('YC', color)]
        colored_reads.write(r)

    colored_reads.close()
    pysam.sort("-o", out_file, 'tmp.bam')
    pysam.index(out_file)

In [13]:
in_file = '/data/parastou/Star-Lab/Experiments/Interesting-HGsample_AATGTA_CGTCCTGGGG/Alignments3/NStar0.Aligned.out.bam'
#in_file = '/data/parastou/Star-Lab/Alignments/NStar1.Aligned.out.bam'
#in_file = '/data/parastou/Star-Lab/Alignments/NStar25.Aligned.out.bam'
#in_file = '/data/parastou/Star-Lab/Alignments/NStar3.Aligned.out.bam'

In [14]:
color_reads(in_file)

In [15]:
def bam_to_df(in_file):
    
    st = pysam.AlignmentFile(in_file,"rb")
    st.reset()
    reads = st.fetch(until_eof=True)
    
    nh = 0
    ref = None
    ref_st = None

    rl = []
    for r in reads:
        qn = r.query_name
        if r.has_tag('NH'):
            nh = r.get_tag('NH')
            ref = r.reference_name
            ref_st = r.reference_start
        cigar = r.cigarstring
        props = (qn, ref, ref_st, nh, cigar, ref+':'+str(ref_st))       
        rl.append(props)
        df = pd.DataFrame(rl, columns=['QName','Ref','Start','NH','CIGAR','IGV-MAP'])
    return df

In [16]:
rl1 = bam_to_df(in_file)

In [17]:
gb = rl1.groupby(['Ref'])
groups = dict(list(gb))
starts = sorted(list(groups['chr21']['Start']))

KeyError: 'chr21'

In [49]:
print starts[0]
print starts[-1]

8210158
8988699


In [54]:
from itertools import tee, izip
def gapsofsize(l, length):
    a = sorted(l)[1:]
    b = sorted(l)[:-1]
    it = zip(a,b)
    
    for i,j in zip(a,b):
        if i - j > length:
            print j, i
    #next(b, None)
    #return ( p for x, y in izip(a, b) if y-x < length+1 for p in xrange(x+1,y) )


#print list(gapsofsize(starts, 1500))
gapsofsize(starts, 1500)

8210605 8254473
8254813 8393193
8393640 8437403
8437850 8988699


In [55]:
print min(starts)

8210158


In [18]:
for qn, group in rl1.groupby(['QName']):
    print qn
    print group
    print '-' * 80

L183:338:CAGAAANXX:4:1101:5182:44104
                                   QName   Ref     Start  NH CIGAR  \
10  L183:338:CAGAAANXX:4:1101:5182:44104  chrM      1767   4   50M   
11  L183:338:CAGAAANXX:4:1101:5182:44104  chr2  33767471   4   50M   
12  L183:338:CAGAAANXX:4:1101:5182:44104  chr5  80650900   4   50M   
13  L183:338:CAGAAANXX:4:1101:5182:44104  chr3  96618088   4   50M   

          IGV-MAP  
10      chrM:1767  
11  chr2:33767471  
12  chr5:80650900  
13  chr3:96618088  
--------------------------------------------------------------------------------
L183:338:CAGAAANXX:4:1116:2253:94090
                                    QName   Ref     Start  NH  CIGAR  \
219  L183:338:CAGAAANXX:4:1116:2253:94090  chrM      1776   3    50M   
220  L183:338:CAGAAANXX:4:1116:2253:94090  chr5  80650891   3    50M   
221  L183:338:CAGAAANXX:4:1116:2253:94090  chr3  96618082   3  3S47M   

           IGV-MAP  
219      chrM:1776  
220  chr5:80650891  
221  chr3:96618082  
---------------------

In [19]:
for ref, group in rl1.groupby(['Ref']):
    print ref
    print group
    print '-' * 80

chr1
                                    QName   Ref      Start  NH   CIGAR  \
59   L183:338:CAGAAANXX:4:2314:3274:51709  chr1   14445252  90  17S33M   
63   L183:338:CAGAAANXX:4:2314:3274:51709  chr1   89036989  90  17S33M   
66   L183:338:CAGAAANXX:4:2314:3274:51709  chr1  161380531  90  33M17S   
76   L183:338:CAGAAANXX:4:2314:3274:51709  chr1   43060711  90  33M17S   
100  L183:338:CAGAAANXX:4:2314:3274:51709  chr1   87004148  90  33M17S   
126  L183:338:CAGAAANXX:4:2314:3274:51709  chr1  156478757  90  18S32M   

            IGV-MAP  
59    chr1:14445252  
63    chr1:89036989  
66   chr1:161380531  
76    chr1:43060711  
100   chr1:87004148  
126  chr1:156478757  
--------------------------------------------------------------------------------
chr10
                                    QName    Ref      Start  NH   CIGAR  \
89   L183:338:CAGAAANXX:4:2314:3274:51709  chr10   72164188  90  17S33M   
114  L183:338:CAGAAANXX:4:2314:3274:51709  chr10   30452227  90  32M18S   
115  L183:

In [60]:
in_file = '/data/parastou/Star-Lab/Experiments/Complete-HGsample_AATGTA_AGCTCTTGTA/Alignments/NStar1.Aligned.out.bam'
chr_name = 'chr21'
xm = 'AGCTCTTGTA' 

r1 = [8210158 - 1500, 8210605 + 1500]
r2 = [8254473 - 1500, 8254813 + 1500]
r3 = [8393193 - 1500, 8393640 + 1500]
r4 = [8437403 - 1500, 8437850 + 1500]

-----------------------------

In [117]:
st = pysam.AlignmentFile('/data/parastou/Star-Lab/test/NStar10.Aligned.sorted.out.tagged.bam','rb')
reads = st.fetch(until_eof=True)

In [118]:
qn_mm = {}
for r in reads:
    qn = r.query_name
    nm = r.get_tag('nM')
    if qn in qn_mm:
        qn_mm[qn].append(nm)
    else:
        qn_mm.update({qn:[nm]})

In [119]:
qn_mm

{'L183:338:CAGAAANXX:4:1312:6838:43650': [0],
 'L183:338:CAGAAANXX:4:1101:20964:65331': [0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:5:1214:3586:9001': [0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:5:2205:18443:21429': [0, 0],
 'L183:338:CAGAAANXX:4:2211:19733:9266': [0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:4:2305:7629:86971': [0, 0, 0, 0],
 'L183:338:CAGAAANXX:4:1201:10912:31654': [0],
 'L183:338:CAGAAANXX:4:1206:16342:57009': [0],
 'L183:338:CAGAAANXX:6:2308:6443:62426': [0],
 'L183:338:CAGAAANXX:5:1316:9239:83420': [3],
 'L183:338:CAGAAANXX:4:1213:16406:70075': [0],
 'L183:338:CAGAAANXX:6:2303:1760:40993': [0, 0],
 'L183:338:CAGAAANXX:6:2112:15280:67543': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:5:1111:8133:73483': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:4:1208:2010:29559': [0, 0, 0, 0, 0, 0, 0, 0],
 'L183:338:CAGAAANXX:5:1204:16448:42011': [0],
 'L183:338:CAGAAANXX:6:2208:5310:71868': [0],
 'L183:338:CAGAAANXX:4:2112:17650:84667': [0, 0, 0, 0, 0, 0, 0, 0

In [90]:
st.reset()
reads = st.fetch(until_eof=True)

Mnames = []
for r in reads:
    if r.reference_name == 'chrM':
        if not r.query_name in Mnames:
            Mnames.append(r.query_name)

In [92]:
d = [item for item in qnames if not item in Mnames]

In [93]:
d

['L183:338:CAGAAANXX:4:2314:3274:51709']

In [97]:
st.reset()
reads = st.fetch(until_eof=True)

for r in reads:
    if r.query_name== 'L183:338:CAGAAANXX:4:2314:3274:51709':
        print r.query_name
        print r.reference_name + ':' + str(r.reference_start)
        print r.query_sequence
        print r.cigarstring
        print

L183:338:CAGAAANXX:4:2314:3274:51709
chr6:123984129
GTCTGGTCTCAGGCTCTTGACCTCAGGTGATCCGCCCATCTCAGCCTCCC
13S37M

L183:338:CAGAAANXX:4:2314:3274:51709
chr20:5568662
GGGAGGCTGAGATGGGCGGATCACCTGAGGTCAAGAGCCTGAGACCAGAC
37M13S

L183:338:CAGAAANXX:4:2314:3274:51709
chr4:87499097
GTCTGGTCTCAGGCTCTTGACCTCAGGTGATCCGCCCATCTCAGCCTCCC
12S38M



In [94]:
duo =['L183:338:CAGAAANXX:5:2216:18847:51121', 'L183:338:CAGAAANXX:5:2304:9982:52232']

In [96]:
st.reset()
reads = st.fetch(until_eof=True)

for r in reads:
    if r.query_name in duo:
        print r.query_name
        print r.reference_name + ':' + str(r.reference_start)
        print r.query_sequence
        print r.cigarstring
        print

L183:338:CAGAAANXX:5:2216:18847:51121
chrM:1766
TCCCTACACGACGCGGGGGCGCAATAGATATAGTACCGCAAGGGAAAGAT
17S33M

L183:338:CAGAAANXX:5:2216:18847:51121
chr2:33767489
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2216:18847:51121
chr11:10509059
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2216:18847:51121
chr5:80650918
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2216:18847:51121
chr3:96618106
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2304:9982:52232
chrM:1766
TCCCTACACGACGCGGGGGCGCAATAGATATAGTACCGCAAGGGAAAGAT
17S33M

L183:338:CAGAAANXX:5:2304:9982:52232
chr2:33767489
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2304:9982:52232
chr11:10509059
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CAGAAANXX:5:2304:9982:52232
chr5:80650918
ATCTTTCCCTTGCGGTACTATATCTATTGCGCCCCCGCGTCGTGTAGGGA
33M17S

L183:338:CA

----------------------------

In [96]:
# method for producing gene name to biotype map (gmap) file
with open('/home/parastou/Desktop/data/parastou/UMI/data/HG/Homo_sapiens.GRCh38.84_chrsNamesUCSC.gtf') as gtf:
    with open('/data/parastou/UMI/data/HG/Homo_sapiens.GRCh38.84_chrsNamesUCSC.gmap','wb') as gmap:
        for line in gtf:
            gene = line.split('\t')[8].split(' ')[1].rstrip('\n').rstrip(';')
            p = line.split('\t')[8].find('gene_biotype ')
            btype = line.split('\t')[8][p+13:].split(' ')[0].rstrip('\n').rstrip(';')
            gmap.write('%s\t%s\n' %(gene,btype))

---------------------------------------

In [32]:
import itertools

def hamming(str1, str2):
    return sum(itertools.imap(str.__ne__, str1, str2))

In [7]:
hamming('GGTCTGAAGTTCAAATCACAGCAACCACATGGTGGCTCACAACCATCTGT','GGTCTGAAGTTCAAATACAAGCAACCACATGGTGGCTCACAACCATCTGT')

3

----------------------------